In [1]:
## Base features
import pandas as pd
mdf = pd.read_csv("../Common/filtered_final_movies_5.tsv", sep="\t")
amdf = pd.read_csv("../Common/additional_movies.tsv", sep="\t")

mdl = pd.Series(mdf['directors'].str.split(',').explode().unique()).tolist()

ddf = pd.DataFrame(columns=['nconst'])
ddf['nconst'] = mdl

mmdf = amdf[['tconst', 'directors', 'startYear', 'worldwide','profit']].copy()
mmdf['directors'] = mmdf['directors'].str.split(',')
mmdf = mmdf.explode('directors')
mmdf.reset_index(drop=True, inplace=True)
mmdf.rename(columns={'directors': 'nconst'}, inplace=True)

director_summary = mmdf.groupby('nconst').agg(
    base_total_gross=('worldwide', 'sum'),
    base_year=('startYear', 'min'),
    base_num_movies=('tconst', 'nunique'),
    base_total_profit = ('profit','sum')
).reset_index()

merged = pd.merge(ddf,director_summary,how="left")
merged["base_num_movies"] = merged["base_num_movies"].fillna(0).astype(int)
merged["base_total_gross"] = merged["base_total_gross"].fillna(0).astype(int)
merged["base_year"] = merged["base_year"].fillna(-1).astype(int)
merged["base_total_profit"] = merged["base_total_profit"].fillna(0).astype(int)


# Calculate base_nbmovies_revenue correctly
mmdf['has_revenue'] = mmdf['worldwide'] != 0
revenue_counts = mmdf.groupby('nconst')['has_revenue'].sum()
merged['base_nbmovies_revenue'] = merged['nconst'].map(revenue_counts).fillna(0).astype(int)

# Calculate base_nbmovies_profit correctly
mmdf['has_profit'] = ~mmdf['profit'].isna()
profit_counts = mmdf.groupby('nconst')['has_profit'].sum()
merged['base_nbmovies_profit'] = merged['nconst'].map(profit_counts).fillna(0).astype(int)


merged.to_csv("./base.tsv", sep='\t', index=False)

In [3]:
## Features calculation
## Loading base features into a dict
import pandas as pd
mdf = pd.read_csv("../Common/filtered_final_movies_5.tsv", sep='\t')
adf = pd.read_csv("base.tsv", sep='\t')
director_dict = adf.set_index('nconst')[['base_year', 'base_num_movies','base_total_gross','base_total_profit','base_nbmovies_revenue', 'base_nbmovies_profit']] \
                .rename(columns={'base_num_movies': 'curr_num_movies',
                                 'base_total_gross' : 'curr_total_gross',
                                 'base_total_profit': 'curr_total_profit',
                                 'base_nbmovies_revenue': 'curr_nbmovies_revenue',
                                 'base_nbmovies_profit': 'curr_nbmovies_profit'}) \
                .to_dict(orient='index')



## Movie df sorted by release_date
mdf = mdf.sort_values(by='release_date')
mdf['director_max_nb_movies'] = 0
mdf['directors_max_tenure'] = 0
mdf['director_max_total_gross'] = 0
mdf['director_max_total_profit'] = pd.NA
mdf['director_max_avg_gross'] = 0.0
mdf['director_max_avg_profit'] = pd.NA


from tqdm import tqdm

for index, row in tqdm(mdf.iterrows()):
    directors = row["directors"].split(',')

    nb = 0
    ten = 0
    total = 0
    total_pr = -1e18
    avg = 0
    avg_pr = -1e18
    
    for director in directors:
        if director not in director_dict.keys():
            director_dict[director] = {'base_year': row["startYear"], 'curr_num_movies': 0, 'curr_total_gross':0, 'curr_total_profit':0, 'curr_nbmovies_revenue':0, 'curr_nbmovies_profit':0}
        data = director_dict.get(director, {'base_year': row["startYear"], 'curr_num_movies': 0, 'curr_total_gross':0, 'curr_total_profit':0, 'curr_nbmovies_revenue':0, 'curr_nbmovies_profit':0})

        if(data['base_year'] == -1):
            data['base_year'] = row["startYear"]
            director_dict[director]["base_year"] = row["startYear"]
        
        nb = max(nb,data['curr_num_movies'])
        ten = max(ten, row['startYear'] - data['base_year'])
        if(data['curr_nbmovies_revenue']  != 0):
            total = max(total, data['curr_total_gross'])
            avg = max(avg, data['curr_total_gross']/ data['curr_nbmovies_revenue'] )
        if(data['curr_nbmovies_profit']  != 0):
            total_pr = max(total_pr, data['curr_total_profit'])
            avg_pr = max(avg_pr, data['curr_total_profit']/ data['curr_nbmovies_profit'] )
        
        director_dict[director]["curr_num_movies"] += 1
        director_dict[director]["curr_nbmovies_revenue"] += 1
        director_dict[director]["curr_total_gross"] += row['worldwide']
        if(not pd.isna(row['budget'])):
            director_dict[director]["curr_total_profit"] += (row['worldwide'] - row['budget'])     
            director_dict[director]["curr_nbmovies_profit"] +=1
    
    # Populate new fields using the correct data types
    mdf.at[index, 'director_max_nb_movies'] = nb
    mdf.at[index, 'directors_max_tenure'] = ten
    mdf.at[index, 'director_max_total_gross'] = total
    mdf.at[index, 'director_max_avg_gross'] = avg
    if total_pr!=-1e18: 
        mdf.at[index, 'director_max_total_profit'] = total_pr
        mdf.at[index, 'director_max_avg_profit'] = avg_pr


12718it [00:01, 7544.07it/s]


In [4]:
mdf.to_csv("../Common/filtered_final_movies_5.tsv", sep='\t', index=False)